In [ ]:
# %pip install openpyxl
# %pip install xlrd


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simpleNote: you may need to restart the kernel to use updated packages.



In [1]:
import pandas as pd

file_path_41 = 'data/housesales/HPSSA Dataset 41 - Number of residential property sales by LSOA.xls'
file_path_42 = 'data/housesales/hpssadataset42numberofdetachedresidentialpropertysalesbylsoa.xlsx'
file_path_43 = 'data/housesales/hpssadataset43numberofsemidetachedresidentialpropertysalesbylsoa.xls'
file_path_44 = 'data/housesales/hpssadataset44numberofterracedresidentialpropertysalesbylsoa.xls'
file_path_45 = 'data/housesales/hpssadataset45numberofflatsresidentialpropertysalesbylsoa.xlsx'

detached_sales = pd.read_excel(file_path_42, sheet_name='Data',header=5)
semidetached_sales = pd.read_excel(file_path_43, sheet_name='Data',header=5)
terraced_sales = pd.read_excel(file_path_44, sheet_name='Data',header=5)
flat_sales = pd.read_excel(file_path_45, sheet_name='Data',header=5)
total_sales = pd.read_excel(file_path_41, sheet_name='Data',header=5)


In [2]:

def process_sales_data(df):
    # Only keep the LSOA code and LSOA name columns.
    df = df.loc[:, ['LSOA code', 'LSOA name'] + [col for col in df.columns if col not in ['LSOA code', 'LSOA name']]]
    
    # Retain data columns from 1996 onwards
    time_cols = [col for col in df.columns 
                 if any(str(year) in str(col) for year in range(1996, 2023))]
    
    
    cols_to_keep = ['LSOA code', 'LSOA name'] + time_cols
    df = df.loc[:, cols_to_keep]
    return df


detached_sales_clean = process_sales_data(detached_sales)
semidetached_sales_clean = process_sales_data(semidetached_sales)
terraced_sales_clean = process_sales_data(terraced_sales)
flat_sales_clean = process_sales_data(flat_sales)
total_sales_clean = process_sales_data(total_sales)


In [3]:
def standardize_time_cols(df):
    import re
    rename_dict = {}
    for col in df.columns:
        match = re.search(r'ending\s+([A-Za-z]+)\s+(\d{4})', str(col))
        if match:
            month = match.group(1)
            year = match.group(2)
            q = {'Mar':'Q1','Jun':'Q2','Sep':'Q3','Dec':'Q4'}[month]
            rename_dict[col] = f"{year}-{q}"
    df = df.rename(columns=rename_dict)
    return df


detached_sales_clean = standardize_time_cols(detached_sales_clean)
semidetached_sales_clean = standardize_time_cols(semidetached_sales_clean)
terraced_sales_clean = standardize_time_cols(terraced_sales_clean)
flat_sales_clean = standardize_time_cols(flat_sales_clean)
total_sales_clean = standardize_time_cols(total_sales_clean)


In [5]:
def melt_sales(df, value_name):
    return df.melt(
        id_vars=['LSOA code', 'LSOA name'],
        var_name='year_qtr',
        value_name=value_name
    )

total_long = melt_sales(total_sales_clean, 'total_sales')
detached_long = melt_sales(detached_sales_clean, 'detached')
semidetached_long = melt_sales(semidetached_sales_clean, 'semi_detached')
terraced_long = melt_sales(terraced_sales_clean, 'terraced')
flat_long = melt_sales(flat_sales_clean, 'flat')


In [6]:
from functools import reduce

dfs_to_merge = [total_long, detached_long, semidetached_long, terraced_long, flat_long] 
merged_df = reduce(
    lambda left, right: pd.merge(
        left, right, 
        on=['LSOA code', 'LSOA name', 'year_qtr'], 
        how='outer'  
    ), 
    dfs_to_merge
)


In [7]:
merged_df = merged_df.rename(columns={'LSOA code': 'lsoa21cd', 'LSOA name': 'lsoa21nm'})

In [9]:
import geopandas as gpd

gdf_london = gpd.read_file('data/LSOA_21.geojson')

london_lsoa_codes = gdf_london['lsoa21cd'].unique()

In [10]:

london_df = merged_df[merged_df['lsoa21cd'].isin(london_lsoa_codes)].copy()

print('Number of LSOA in London:', len(london_lsoa_codes))
print('Number of rows after filtering:', len(london_df))


Number of LSOA in London: 4994
Number of rows after filtering: 503172


In [12]:
london_df2 = london_df.copy()

# Listing of various types
type_cols = ['detached', 'semi_detached', 'terraced', 'flat']  

# Calculate percentage
for col in type_cols:
    london_df2[f'share_{col}'] = london_df2[col] / london_df2['total_sales']


In [13]:
keep_cols = ['lsoa21cd', 'lsoa21nm', 'year_qtr', 'total_sales'] + [f'share_{col}' for col in type_cols]
london_df2 = london_df2[keep_cols]

In [15]:
london_df2.to_csv('data/housesales/property_prop.csv')

In [16]:
file_path_population = 'data/population/lsoapopulationdensity2011-2022.xlsx'

population = pd.read_excel(file_path_population, sheet_name='Mid-2011 to mid-2022 LSOA 2021',header=3)

population = population.drop(columns=['LAD 2021 Code', 'LAD 2021 Name'], errors='ignore')

In [17]:

london_population = population[population['LSOA 2021 Code'].isin(london_lsoa_codes)].copy()

print('Number of LSOA in London:', len(london_lsoa_codes))
print('Number of rows after filtering:', len(london_population))


Number of LSOA in London: 4994
Number of rows after filtering: 4994


In [18]:
london_population = london_population.rename(columns={'LSOA 2021 Code': 'lsoa21cd', 'LSOA 2021 Name': 'lsoa21nm'})

In [20]:
london_population.columns = london_population.columns.str.strip()  

# Only retain the columns for total area and total population.
cols_to_keep = ['lsoa21cd'	,'lsoa21nm', 'Area Sq Km']  
cols_to_keep += [col for col in london_population.columns if 'Population' in col and 'People per Sq Km' not in col]

population_clean = london_population[cols_to_keep].copy()

In [22]:
population_clean.to_csv('data/population/LSOApopulation.csv')

In [27]:
import numpy as np
# 1. Read
df = pd.read_csv('data/population/LSOApopulation.csv')

# 2. Find all population years columns
pop_cols = [col for col in df.columns if 'Population' in col]

# 3. Construct all listings from 1996 to 2022.
all_years = list(range(1996, 2023))
all_pop_cols = [f'Mid-{year}: Population' for year in all_years]

# 4. Fill in missing years
for col in all_pop_cols:
    if col not in df.columns:
        df[col] = np.nan

# 5. Ensure that the column order is the primary key + population by year.
main_cols = [c for c in df.columns if c not in all_pop_cols]
df = df[main_cols + all_pop_cols]

# 6. Horizontal linear interpolation + extrapolation at both ends
def interpolate_row(row):
    pop_data = row[all_pop_cols]
    interp = pop_data.interpolate(method='linear', limit_direction='both')
    return interp

df[all_pop_cols] = df[all_pop_cols].apply(interpolate_row, axis=1)


print(df[['lsoa21cd'] + all_pop_cols[:5]])  # 前5年


       lsoa21cd  Mid-1996: Population  Mid-1997: Population  \
0     E01000001                1472.0                1472.0   
1     E01000002                1438.0                1438.0   
2     E01000003                1348.0                1348.0   
3     E01000005                 987.0                 987.0   
4     E01000006                1731.0                1731.0   
...         ...                   ...                   ...   
4989  E01035718                3192.0                3192.0   
4990  E01035719                1100.0                1100.0   
4991  E01035720                1009.0                1009.0   
4992  E01035721                2456.0                2456.0   
4993  E01035722                2707.0                2707.0   

      Mid-1998: Population  Mid-1999: Population  Mid-2000: Population  
0                   1472.0                1472.0                1472.0  
1                   1438.0                1438.0                1438.0  
2                   1348

In [28]:
df.to_csv('data/population/LSOApopulation_1996_2022.csv')

In [29]:
# 1. Read
pop_df = pd.read_csv('data/population/LSOApopulation_1996_2022.csv')

pop_cols = [col for col in pop_df.columns if 'Population' in col and 'Mid-' in col]

# 2. Wide table to long table
pop_long = pd.melt(
    pop_df,
    id_vars=[col for col in pop_df.columns if col not in pop_cols],   # 保留所有非人口列
    value_vars=pop_cols,
    var_name='year',
    value_name='population'
)

# 3. Extract the year as a number
pop_long['year'] = pop_long['year'].str.extract(r'Mid-(\d{4})').astype(int)


# pop_long.to_csv('data/population/LSOApopulation_long_1996_2022.csv', index=False)


In [31]:
# Generate data for each LSOA for each of the four quarters of each year.
quarters = ['Q1', 'Q2', 'Q3', 'Q4']

# Construct all quarters of the year
pop_long_expand = (
    pop_long
    .assign(key=1)
    .merge(pd.DataFrame({'quarter': quarters, 'key': 1}), on='key')
)

pop_long_expand['year_qtr'] = pop_long_expand['year'].astype(str) + '-' + pop_long_expand['quarter']
pop_long_expand = pop_long_expand.drop(columns=['key', 'quarter'])

# Only retain fields necessary for merging, ensuring consistency with the primary key of the housing ratio data.
cols_needed = ['lsoa21cd', 'lsoa21nm', 'year_qtr', 'population', 'Area Sq Km']
pop_long_expand = pop_long_expand[cols_needed]

pop_long_expand

,lsoa21cd,lsoa21nm,year_qtr,population,Area Sq Km
0,E01000001,City of London 001A,1996-Q1,1472.0,0.1298
1,E01000001,City of London 001A,1996-Q2,1472.0,0.1298
2,E01000001,City of London 001A,1996-Q3,1472.0,0.1298
3,E01000001,City of London 001A,1996-Q4,1472.0,0.1298
4,E01000002,City of London 001B,1996-Q1,1438.0,0.2283
...,...,...,...,...,...
539347,E01035721,Westminster 023H,2022-Q4,2236.0,0.2169
539348,E01035722,Westminster 024G,2022-Q1,2123.0,0.1222
539349,E01035722,Westminster 024G,2022-Q2,2123.0,0.1222
539350,E01035722,Westminster 024G,2022-Q3,2123.0,0.1222


In [32]:
prop_df = pd.read_csv('data/housesales/property_prop.csv')

merged = prop_df.merge(
    pop_long_expand,
    on=['lsoa21cd', 'lsoa21nm', 'year_qtr'],
    how='left'
)

merged


,Unnamed: 0,lsoa21cd,lsoa21nm,year_qtr,total_sales,share_detached,share_semi_detached,share_terraced,share_flat,population,Area Sq Km
0,0,E01000001,City of London 001A,1996-Q1,69,0.000000,0.000000,0.000000,1.000000,1472.0,0.1298
1,1,E01000001,City of London 001A,1996-Q2,92,0.000000,0.000000,0.000000,1.000000,1472.0,0.1298
2,2,E01000001,City of London 001A,1996-Q3,127,0.000000,0.000000,0.000000,1.000000,1472.0,0.1298
3,3,E01000001,City of London 001A,1996-Q4,140,0.000000,0.000000,0.000000,1.000000,1472.0,0.1298
4,4,E01000001,City of London 001A,1997-Q1,147,0.000000,0.000000,0.006803,0.993197,1472.0,0.1298
...,...,...,...,...,...,...,...,...,...,...,...
503167,3544771,E01033746,Greenwich 038E,2021-Q4,26,0.038462,0.000000,0.653846,0.230769,1514.0,0.1270
503168,3544772,E01033746,Greenwich 038E,2022-Q1,27,0.037037,0.037037,0.740741,0.111111,1529.0,0.1270
503169,3544773,E01033746,Greenwich 038E,2022-Q2,20,0.050000,0.050000,0.650000,0.250000,1529.0,0.1270
503170,3544774,E01033746,Greenwich 038E,2022-Q3,24,0.041667,0.041667,0.666667,0.208333,1529.0,0.1270


In [33]:
merged = merged.drop(columns={'Unnamed: 0'})

In [34]:
merged.to_csv('data/pop_prop.csv', index=False)